In [1]:
#install selenium 

!pip install selenium


In [2]:
!pip install mysql-connector-python

In [1]:
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import streamlit as st
import pandas as pd



driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
#driver = webdriver.Firefox()  

def scrape_redbus(source, destination,travel_date):
    driver.get('https://www.redbus.in/')
    time.sleep(5)    
    
    driver.find_element(By.ID, 'src').send_keys(source)
    driver.find_element(By.ID, 'dest').send_keys(destination)   
    
    
    
    travel_date_obj = datetime.strptime(travel_date, '%d-%b-%Y')
    day = travel_date_obj.day
    month_year_str = travel_date_obj.strftime('%d %b %Y')

    
    date_picker = WebDriverWait(driver, 10).until(
       EC.element_to_be_clickable((By.CSS_SELECTOR, '.labelCalendarContainer'))
    )
    date_picker.click()
    
    day_str = travel_date_obj.strftime('%d %b')
    year_str = travel_date_obj.strftime('%Y')

    
    #driver.execute_script(f"document.getElementById('onwardCal').value = '{travel_date}';")
    driver.find_element(By.ID, 'onwardCal').send_keys(travel_date)
    
    
    time.sleep(5)
    
    driver.find_element(By.ID, 'search_button').click()      
    
    
    time.sleep(10)
    
    # Extract data
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')
    data = []
    for bus in buses:        
        route_name = f"{source} - {destination}"
        route_link = driver.current_url
        bus_name = bus.find_element(By.CLASS_NAME, 'travels').text
        bus_type = bus.find_element(By.CLASS_NAME, 'bus-type').text
        departure_time = bus.find_element(By.CLASS_NAME, 'dept-time').text
        duration = bus.find_element(By.CLASS_NAME, 'duration').text
        arrival_time = bus.find_element(By.CLASS_NAME, 'reach-time').text
        star_rating = bus.find_element(By.CLASS_NAME, 'rating').text
        price = bus.find_element(By.CLASS_NAME, 'fare').text.replace('₹', '').replace(',', '')
        seats_available = bus.find_element(By.CLASS_NAME, 'seat-left').text

        # Convert times to datetime
        try:
            departure_time = datetime.strptime(departure_time, '%I:%M %p')
            arrival_time = datetime.strptime(arrival_time, '%I:%M %p')
        except ValueError:
            departure_time = None
            arrival_time = None

        data.append((route_name, route_link, bus_name, bus_type, departure_time, duration, arrival_time, float(star_rating), float(price), int(seats_available)))
        
        # Inserting data into the database
    conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='12345678',
    database='redbus_db'
    )
    cursor = conn.cursor()
    cursor.executemany('''
    INSERT INTO bus_routes (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', data)
    conn.commit()
    conn.close()
    
#
scrape_redbus('Bangalore', 'Chennai','27-Jul-2024')
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF63976EEB2+31554]
	(No symbol) [0x00007FF6396E7EE9]
	(No symbol) [0x00007FF6395A872A]
	(No symbol) [0x00007FF63957D995]
	(No symbol) [0x00007FF6396244D7]
	(No symbol) [0x00007FF63963C051]
	(No symbol) [0x00007FF63961CDD3]
	(No symbol) [0x00007FF6395EA33B]
	(No symbol) [0x00007FF6395EAED1]
	GetHandleVerifier [0x00007FF639A78B2D+3217341]
	GetHandleVerifier [0x00007FF639AC5AF3+3532675]
	GetHandleVerifier [0x00007FF639ABB0F0+3489152]
	GetHandleVerifier [0x00007FF63981E786+750614]
	(No symbol) [0x00007FF6396F376F]
	(No symbol) [0x00007FF6396EEB24]
	(No symbol) [0x00007FF6396EECB2]
	(No symbol) [0x00007FF6396DE17F]
	BaseThreadInitThunk [0x00007FFD528A257D+29]
	RtlUserThreadStart [0x00007FFD5338AF28+40]
